In [2]:
from week1 import box3d, PiInv, Pi,project_points,plot3d
import numpy as np 
import matplotlib.pyplot as plt 
from sympy import Matrix

Exercise 2.1
Reuse the box3d function from last week. Assume that f = 600, α = 1, β = 0, and δx = δy = 400.
Given a traditional camera, what is the resolution in pixels?
Also assume R = I, and t = [0, .2, 1.5]T. Use projectpoints from last week, to project the
points.

In [11]:
#Define the given parameters 

Q = box3d()
f = 600
beta = 0
alpha = 1 
dx = 400
dy = 400

K = np.array([[f,f*beta,dx],[0,alpha*f,dy],[0,0,1]])
R = np.eye(3)
t = np.array([0,0.2,1.5])

#The projected points 
p = project_points(K,R,t,Q)

Are all the points captured by the image sensor?


In [4]:
#The projection matrix 

P = K@np.column_stack((R,t))
Matrix(P)

⎡600.0    0    400.0  600.0⎤
⎢                          ⎥
⎢  0    600.0  400.0  720.0⎥
⎢                          ⎥
⎣  0      0     1.0    1.5 ⎦

2 x 400 = 800 pixels along each dimensions, i.e. 800 x 800.

Where does the corner P 1 = [−0.5, −0.5, −0.5] project to?


In [5]:
corner = np.array([-0.5,-0.5,-0.5])

project_points(K,R,t,corner)

array([100., 220.])

The corner projects to [100,220].T

#### Create a new or change your function projectpoints to a version that also takes distCoeffs as
#### an input. The list distCoeffs should contain the distortion coefficients [k3, k5, k7, . . . ]. Make the function work for at least 3 coefficients.


In [165]:
def PiInv(coordinate):
    """input: inhom coordinates
    returns: homogenous coordinates"""  

    if (coordinate.shape[0])>1:
        w = np.ones((1,coordinate.shape[1]))
        hom = np.vstack((coordinate,w))
        print("i was here")

    else:
        w = 1
        hom = np.append(coordinate,w)

    return hom 

In [240]:
from sympy import Matrix, pretty_print


def project_points_distort(A_p,A_q,R,t,Q,distCoeffs=None):


    if distCoeffs is not None:
      



      p_d = A_p @  np.hstack((R,t)) @ Q

        #r_i = np.sqrt(np.sum(pi_d**2,axis=0))
        
      #  powers = np.arange(2,len(distCoeffs)*2+1,2)
      #  delta_r_i = np.array([distCoeffs]) @ np.power(r_i,powers.reshape(-1,1))

      #  p_c = pi_d * (1+delta_r_i.T)

     #   print(PiInv(p_c).T.shape)
     #   qi = K @ PiInv(p_c).T
      #  qi = Pi(qi)


      return 2

 


Q = box3d()

f = 600
beta = 0
alpha = 1 
dx = 400
dy = 400

A_p = np.array([[f,0,0],[0,f,0],[0,0,1]])
A_q = np.array([[1,beta,dx],[0,alpha,dy],[0,0,1]])


R = np.eye(3)
t = np.array([0,0.2,1.5]).reshape(-1,1)
points = np.array([-0.5,-0.5,-0.5])

print(A_p.shape,R.shape,t.shape)

project_points_distort(A_p,A_q,R,t,points,np.array([-2,0,0]))

(3, 3) (3, 3) (3, 1)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 4)

In [237]:
R = np.array([[0.9887,-0.0004,0.1500],[0.0008,1,-0.0030],[-0.15,0.0031,0.9887]])
t = np.array([-2.1811,0.0399,0.5072]).reshape(-1,1)
A = np.array([[2774.5,0,806.8],[0,2774.5,622.6],[0,0,1]])


A_p = np.array([[2774.5,0,0],[0,2774.5,0],[0,0,1]])
A_q = np.array([[1,0,806.8],[0,1,622.6],[0,0,1]])

#P = A @ np.hstack((R,t))
Q = np.array([-1.3540,0.5631,8.8734,1]).T




p_d = A_p @  np.hstack((R,t)) @ Q

r_i = np.sqrt(np.sum(Pi(p_d)**2,axis=0))
distCoeffs = np.array([-5.1806e-8,1.4192e-15])
powers = np.arange(2,len(distCoeffs)*2+1,2)
delta_r_i = np.array([distCoeffs]) @ np.power(r_i,powers.reshape(-1,1))
p_c = Pi(p_d) * (1+delta_r_i)
p_c = PiInv(np.array(p_c))
q = A_q @ p_c
Matrix(q)


print(A_p.shape,R.shape,t.shape)


(3, 3) (3, 3) (3, 1)


In [215]:
A.shape

In [207]:
p_c.shape[2]

IndexError: tuple index out of range

In [ ]:
        r_i = np.sqrt(np.sum(pi_d**2,axis=0))
        
        powers = np.arange(2,len(distCoeffs)*2+1,2)
        delta_r_i = np.array([distCoeffs]) @ np.power(r_i,powers.reshape(-1,1))



In [120]:
Matrix(dot_product_sum)

⎡k₁⋅r₁ + k₂⋅r₁⎤
⎢             ⎥
⎢k₁⋅r₂ + k₂⋅r₂⎥
⎢             ⎥
⎢k₁⋅r₃ + k₂⋅r₃⎥
⎢             ⎥
⎣k₁⋅r₄ + k₂⋅r₄⎦